Notebook para la conversión de db sqlite to pg

In [36]:
import sqlite3
import pandas as pd
import psycopg2
from configparser import ConfigParser

Bloque 1: Sqlite Dump File or dataframe

In [44]:
database = "D:/BDD-Sqlite(V1)/app.db"
conn = sqlite3.connect(database)

In [49]:
dbSchemaQueries = pd.read_sql("select sql from sqlite_master", con=conn)
dbSchemaQueries = dbSchemaQueries.mask(dbSchemaQueries.eq(None)).dropna()
tableNames = pd.read_sql("SELECT name FROM sqlite_master WHERE (type='table')", con=conn)
dbData = []
selectQuery = "SELECT * from "
for i in range(len(tableNames)):
    currentTable = tableNames['name'][i]
    dbData.append(pd.read_sql_query(selectQuery + currentTable, con=conn))

selectQuery = "SELECT * from "
tableName= tableNames['name'][1]
print(dbData)

[    version_num
0  4474fcbaf5d2,     id                                               name  \
0    1  ORDEN DE ESTUDIOS CORPORATIVOS POR FECHA (PRIM...   
1    2  EN COMPRAS PONER UN DESPLEGABLE DE QUE LABORAT...   
2    3  CUANDO QUIERO EDITAR COMPRAS (ITEM ESTADO INTE...   
3    4  AGREGAR UN GESTIONADOR DE TOMA DE MUESTRA A DO...   
4    5  aumentar cantidad de caracteres en campos desc...   
5    6                    proveedor en items de productos   
6    7                      cantidad en items de producto   
7    8    corregir default empresa en estudio corporativo   
8    9                                 STOCK DE PRODUCTOS   
9   10                                       NUEVA COMPRA   
10  11                                            COMPRAS   

                 creation_date                  final_date  \
0   2020-07-05 21:55:07.986928  2020-07-11 10:19:08.050227   
1   2020-07-05 22:07:51.040644                        None   
2   2020-07-05 22:09:46.543876  2020-07-11 10:2

In [46]:
#for i in dbSchemaQueries.index:
#    print(dbSchemaQueries['sql'][i])
#print(dbSchemaQueries['sql'][0].index('a'))
a = dbSchemaQueries['sql'][12].replace('CHECK', '@')
b = a.replace(a[a.index('@'):len(a)-1], '')
temp = list(b)
temp[len(temp)-5] = ''
b = "".join(temp)
print(dbSchemaQueries['sql'][12])
print(b[:b.index(b[len(b)-5])] + '' + b[b.index(b[len(b)-5])+1:])
print(b)

CREATE TABLE warehouse (
	id INTEGER NOT NULL, 
	name VARCHAR(16), 
	location VARCHAR(16), 
	notes VARCHAR(255), 
	soft_delete BOOLEAN, 
	PRIMARY KEY (id), 
	CHECK (soft_delete IN (0, 1))
)
CREATE TABLE warehouse (
	id INTEGER NOT NULL, 
	name VARCHAR(16, 
	location VARCHAR(16), 
	notes VARCHAR(255), 
	soft_delete BOOLEAN, 
	PRIMARY KEY (id) 
	)
CREATE TABLE warehouse (
	id INTEGER NOT NULL, 
	name VARCHAR(16), 
	location VARCHAR(16), 
	notes VARCHAR(255), 
	soft_delete BOOLEAN, 
	PRIMARY KEY (id) 
	)


In [20]:
conn.close()

Bloque 2: Dump file or dataframe parse

In [47]:
#dbSchemaQueries.replace('DATETIME', 'TIMESTAMP')
parsed_dbSchemaQueries = []
for i in dbSchemaQueries.index:
#    if(aux.find('CHECK') == -1):
    aux = dbSchemaQueries['sql'][i]
    aux = aux.replace('DATETIME', 'TIMESTAMP')
    aux = aux.replace('user', 'user_')
    if(aux.find('CHECK') != -1):
        a = aux.replace('CHECK', '@')
        b = a.replace(a[a.index('@'):len(a)-1], '')
        temp = list(b)
        temp[len(temp)-5] = ''
        aux = "".join(temp)
    parsed_dbSchemaQueries.append(aux)


print(parsed_dbSchemaQueries)

['CREATE TABLE alembic_version (\n\tversion_num VARCHAR(32) NOT NULL, \n\tCONSTRAINT alembic_version_pkc PRIMARY KEY (version_num)\n)', 'CREATE TABLE feedback (\n\tid INTEGER NOT NULL, \n\tname VARCHAR(128), \n\tcreation_date TIMESTAMP, \n\tfinal_date TIMESTAMP, \n\tnotes TEXT, \n\tanswer TEXT, \n\tuser_name VARCHAR(16), \n\tPRIMARY KEY (id)\n)', 'CREATE TABLE item_category (\n\tid INTEGER NOT NULL, \n\tname VARCHAR(32), \n\tPRIMARY KEY (id)\n)', 'CREATE UNIQUE INDEX ix_item_category_name ON item_category (name)', 'CREATE TABLE organization_category (\n\tid INTEGER NOT NULL, \n\tname VARCHAR(32), \n\tPRIMARY KEY (id)\n)', 'CREATE UNIQUE INDEX ix_organization_category_name ON organization_category (name)', 'CREATE TABLE organization_client_category (\n\tid INTEGER NOT NULL, \n\tname VARCHAR(32), \n\tPRIMARY KEY (id)\n)', 'CREATE UNIQUE INDEX ix_organization_client_category_name ON organization_client_category (name)', 'CREATE TABLE user_ (\n\tid INTEGER NOT NULL, \n\tuser_name VARCHAR(1

Bloque 3: Creación de base de datos

Bloque 4: Dump file to pg

In [41]:
def config(archivo='pg.ini', seccion='postgresql'):
    # Crear el parser y leer el archivo
    parser = ConfigParser()
    parser.read(archivo)
 
    # Obtener la sección de conexión a la base de datos
    db = {}
    if parser.has_section(seccion):
        params = parser.items(seccion)
        for param in params:
            db[param[0]] = param[1]
        return db
    else:
        raise Exception('Secccion {0} no encontrada en el archivo {1}'.format(seccion, archivo))

In [22]:
def conectar():
    """ Conexión al servidor de bases de datos PostgreSQL """
    conexion = None
    try:
        # Lectura de los parámetros de conexion
        params = config()
 
        # Conexion al servidor de PostgreSQL
        print('Conectando a la base de datos PostgreSQL...')
        print(params)
        conexion = psycopg2.connect(**params)
        conexion.autocommit = True
 
        # creación del cursor
        cur = conexion.cursor()
        
        # Ejecución de una consulta con la version de PostgreSQL
        print('La version de PostgreSQL es la:')
        cur.execute('SELECT version()')
 
        # Ahora mostramos la version
        version = cur.fetchone()
        print(version)
       
       # Ejecutamos una consulta
       ## cur.execute( "SELECT nombre, apellidos FROM empleados" )

        # Recorremos los resultados y los mostramos
        ##for nombre, apellidos in cur.fetchall() :
        ##    print (nombre, apellidos)

        # Cierre de la comunicación con PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conexion is not None:
            """conexion.close()
            print('Conexión finalizada.')"""

In [42]:
conectar()

Conectando a la base de datos PostgreSQL...
{'host': '192.168.1.105', 'database': 'Prueba2', 'user': 'gdeluca', 'password': 'gdeluca', 'port': '5432'}
La version de PostgreSQL es la:
('PostgreSQL 14.2 (Ubuntu 14.2-1ubuntu1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.2.0-19ubuntu1) 11.2.0, 64-bit',)


In [48]:
# Creamos el cursor con el objeto conexion
params = config()
conexion = psycopg2.connect(**params)
conexion.autocommit = True
cur = conexion.cursor()
# Ejecutamos una consulta
for i in range(len(parsed_dbSchemaQueries)):
    cur.execute(parsed_dbSchemaQueries[i])

# Recorremos los resultados y los mostramos
"""for id, name in cur.fetchall() :
    print(id, name)"""

# Cerramos la conexión
conexion.commit()
conexion.close()

DuplicateTable: relation "alembic_version" already exists
